In [19]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types
import os
import asyncio

In [20]:
try:
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  print("Authentication & API setup completed.")
except Exception as e:
  print(f"Errror: {e}")

Authentication & API setup completed.


In [21]:
# retry_options

retry_config = types.HttpRetryOptions(
  attempts=5,
  initial_delay=1,
  exp_base=7,
  http_status_codes=[429,500,503,504]
)

### Helper function

Helper function that manages a complete conversation session, handling session
creation/retrieval, query processing, and response streaming. It supports
both single queries and multiple queries in sequence.

Example:

```
>>> await run_session(runner, "What is the capital of France?", "geography-session")
>>> await run_session(runner, ["Hello!", "What's my name?"], "user-intro-session")
```

In [22]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name : str ="default"

):
  print(f"\n ### SESSION: {session_name}")

  #Get APP name from the Runner
  app_name = runner_instance.app_name

  #Attempt to create a new session or retrieve an existing one
  try:
    session = await session_service.create_session(
      app_name=app_name, user_id=USER_ID, session_id=session_name
    )
  except:
    session= await session_service.get_session(
      app_name=app_name,user_id =USER_ID,session_id=session_name
    )

  #Process queries if provided
  if user_queries:
    #convert single query to list for uniform processing
    if type(user_queries)==str:
      user_queries=[user_queries]

    #Process each query in the list sequentially
    for query in user_queries:
      print(f"\n User > {query}")

      #Convert the query string to the ADK content format
      query=types.Content(role="user",parts=[types.Part(text=query)])

      #stream the agents response asynchronously
      async for event in runner_instance.run_async(
        user_id=USER_ID, session_id=session.id, new_message=query
      ):
        #check if the event contains valid content
        if event.content and event.content.parts:
          #filter out empty of None responses before printing
          if(
            event.content.parts[0].text !="None"
            and event.content.parts[0].text
          ):
            print(f"{MODEL_NAME} > ", event.content.parts[0].text)
  else:
    print("No queries!")

In [23]:
# Implementing Stateful Agent
#he can remember and have constructive conversations.

APP_NAME = "default" #Application
USER_ID = "default" #User
SESSION = "default" #Session
MODEL_NAME = "gemini-2.5-flash-lite"

#step1 :create llm agent

root_agent = Agent(
  model=Gemini(model=MODEL_NAME,retry_options=retry_config),
  name="text_chat_bot",
  description="A text chatbot", #description of the agents purpose
)

#Step2: set up session management
#InMemorySessionService stores conversations in RAM temporary.

session_service = InMemorySessionService()

#Step3: create the runner
runner=Runner(agent=root_agent,app_name=APP_NAME,session_service=session_service)

print("Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

App name mismatch detected. The runner is configured with app name "default", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


In [24]:
### Testing the Stateful agent
await run_session(
  runner,
  [
    "Hi,Im Ilias, whats the climate change Risk?",
    "Whats my name?",#this time , the agent should remember
  ],
  "stateful-agentic-session"
)


 ### SESSION: stateful-agentic-session

 User > Hi,Im Ilias, whats the climate change Risk?
gemini-2.5-flash-lite >  Hi Ilias! It's great to meet you.

Climate change risk refers to the **potential for negative impacts and losses that arise from the changing climate**. It's not just about the weather getting warmer, but about the wide-ranging consequences that this warming can have on our planet and on human society.

Here's a breakdown of what climate change risk entails:

**1. Physical Risks:** These are the direct impacts of a changing climate on the environment and natural systems. They include:

*   **Extreme Weather Events:** More frequent and intense heatwaves, droughts, floods, storms (hurricanes, typhoons), wildfires, and heavy rainfall.
*   **Sea Level Rise:** Leading to coastal flooding, erosion, and saltwater intrusion into freshwater sources.
*   **Changes in Precipitation Patterns:** Causing water scarcity in some regions and increased flooding in others.
*   **Ocean Aci

#### Restarting the kernel, the agent wont remember anything

In [25]:
await run_session(
    runner,
    ["What did I ask you about earlier?", "And remind me, what's my name?"],
    "stateful-agentic-session",
)   #Note, we are using same session name


 ### SESSION: stateful-agentic-session

 User > What did I ask you about earlier?
gemini-2.5-flash-lite >  Earlier, you asked me about "the climate change Risk".

 User > And remind me, what's my name?
gemini-2.5-flash-lite >  Your name is Ilias.


Persistent Sessions with `DatabaseSessionService`

While `InMemorySessionService` is great for prototyping, real-world applications need conversations to survive restarts, crashes, and deployments. Let's level up to persistent storage!

### 3.1 Choosing the Right SessionService

ADK provides different SessionService implementations for different needs:

| Service | Use Case | Persistence | Best For |
|---------|----------|-------------|----------|
| **InMemorySessionService** | Development & Testing | ❌ Lost on restart | Quick prototypes |
| **DatabaseSessionService** | Self-managed apps | ✅ Survives restarts | Small to medium apps |
| **Agent Engine Sessions** | Production on GCP | ✅ Fully managed | Enterprise scale |


In [38]:
# Step 1: Create the same agent (notice we use LlmAgent this time)
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print(" Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

App name mismatch detected. The runner is configured with app name "default", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


 Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [39]:
#verifying that the session data is isolated
await run_session(
  runner,["Whats my name?"],"test-db-session-02"
)


 ### SESSION: test-db-session-02

 User > Whats my name?
gemini-2.5-flash-lite >  I do not have access to your personal information, including your name. I am a text chatbot with persistent memory, but my memory is limited to our conversation history. I cannot access external data or user accounts.


### How events are stored in the Database

Using sqlite DB to store information

In [40]:
import sqlite3

def check_data_in_db():
  with sqlite3.connect("my_agent.db") as connection:
    cursor = connection.cursor()
    result = cursor.execute(
      "select app_name, session_id, author, content from events"
    )
    print(i[0] for i in result.description)
    for result in result.fetchall():
      print(result)


How the events are stored in the database

['app_name', 'session_id', 'author', 'content']

('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Sam! What is the capital of the United States?"}], "role": "user"}')

('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hi Sam! The capital of the United States is Washington, D.C."}], "role": "model"}')

('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')

('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Your name is Sam."}], "role": "model"}')

('default', 'test-db-session-01', 'user', '{"parts": [{"text": "What is the capital of India?"}], "role": "user"}')

('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "The capital of India is New Delhi."}], "role": "model"}')

('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')

('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Your name is Sam."}], "role": "model"}')

('default', 'test-db-session-02', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')

('default', 'test-db-session-02', 'text_chat_bot', '{"parts": [{"text": "I do not have access to your personal information, including your name. I am a large language model, trained by Google."}], "role": "model"}')


### ADKs Context Compaction(ADK's default compaction) to automatically reduce the context that's stored in the Session.

**`EventsCompactionConfig`** defines two key variables:

- **compaction_interval**: Asks the Runner to compact the history after every `n` conversations
- **overlap_size**: Defines the number of previous conversations to retain for overlap

In [41]:
#re-define the app with Events Compaction enabled
research_app_compacting = App(
  name = "research_app_compacting",
  root_agent=chatbot_agent,
  #this is the new part!
  events_compaction_config=EventsCompactionConfig(
    compaction_interval=3, #Trigger Compaction eveny 3 invocations
    overlap_size=1, #keep 1 previous turn for context
  ),
)

db_url= "sqlite:///my_agent_data.db"
session_service=DatabaseSessionService(db_url=db_url)

#Create a new runner for the upgraded app
research_runner_compacting = Runner(
  app=research_app_compacting, session_service=session_service
)


/var/folders/kw/6d_1c0rj22d1pqvkf0fldhlm0000gn/T/ipykernel_56024/3425662949.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(
App name mismatch detected. The runner is configured with app name "research_app_compacting", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


In [42]:
# Turn 1
await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    "compaction_demo",
)

# Turn 2
await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    "compaction_demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    research_runner_compacting,
    "Tell me more about the second development you found.",
    "compaction_demo",
)

# Turn 4
await run_session(
    research_runner_compacting,
    "Who are the main companies involved in that?",
    "compaction_demo",
)


 ### SESSION: compaction_demo

 User > What is the latest news about AI in healthcare?
gemini-2.5-flash-lite >  The field of AI in healthcare is rapidly evolving, so "latest" can be a moving target! However, I can give you a rundown of some of the most significant and ongoing trends and news in AI for healthcare as of my last update:

**Key Areas of Advancement & News:**

*   **Drug Discovery and Development:**
    *   **Accelerated R&D:** AI is dramatically speeding up the identification of potential drug candidates, predicting their efficacy, and optimizing clinical trial design. Companies are using AI to analyze vast biological datasets to find novel targets and design molecules.
    *   **Personalized Medicine:** AI is crucial for developing drugs tailored to an individual's genetic makeup and disease profile. This is leading to more effective treatments with fewer side effects.
    *   **Recent News:** Many pharmaceutical giants are announcing new partnerships with AI drug discov

In [45]:
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)
print(final_session)

id='compaction_demo' app_name='research_app_compacting' user_id='default' state={} events=[Event(model_version=None, content=Content(
  parts=[
    Part(
      text='What is the latest news about AI in healthcare?'
    ),
  ],
  role='user'
), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-2bc2fb93-69b4-424c-b6b8-0b36270067d1', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=None), long_running_tool_ids=set(), branch=None, id='43efaa7c-9f8f-4d8

In [61]:
for event in final_session.events:
  print(event)

model_version=None content=Content(
  parts=[
    Part(
      text='What is the latest news about AI in healthcare?'
    ),
  ],
  role='user'
) grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='e-2bc2fb93-69b4-424c-b6b8-0b36270067d1' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=None) long_running_tool_ids=set() branch=None id='43efaa7c-9f8f-4d85-8291-63423d822aea' timestamp=1763508658.829886
model_version=None content=Content(
  parts=[
    Part(
      text="""T

In [62]:
print("======Searching for Compaction Summary Event =======")
found_summary = False #flag
for event in final_session.events:
  #compaction events have a 'compaction' attribute
  if event.actions and event.actions.compaction:
    print("FOUND THE COMPACTION EVENT:")
    print(F"Author: {event.author}")
    print(f"Compacted Information: {event}")
    found_summary = True
    break

if not found_summary:
  print("No compaction event found.")

======Searching for Compaction Summary Event =======
FOUND THE COMPACTION EVENT:
Author: user
Compacted Information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='87acb603-a521-407e-bbae-4887435cdb1e' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1763508658.829886, 'end_timestamp': 1763508666.789436, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data': None, 't

### Working with Session State

#### Creating custom tools for Session State management
#### Identifying `transferable characteristic` , like user's name and their country, also creating toold to capture and same it.

The username is a perfect example of information that:

- Is introduced once but referenced multiple times
- Should persist throughout a conversation
- Represents a user-specific characteristic that enhances personalization

Here, for demo purposes, i'll create two tools that can store and retrieve user name and country from the Session State. **Note that all tools have access to the `ToolContext` object.**

In [75]:
#Define scope levels for state keys(following best practises)
USER_NAME_SCORE_LEVELS=("temp","user","app")


#This demonstrates how tools can write to session state using tool_context.
#The 'user:' prefix indicates this is user-specific data.

def save_userinfo(
    tool_context: ToolContext, user_name: str, country: str
) -> Dict[str, Any]:
    """
    Tool to record and save user name and country in session state.

    Args:
        user_name: The username to store in session state
        country: The name of the user's country
    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["user:name"]=user_name
    tool_context.state["user:country"]=country
    return {"status": "success"}


#This demonstrates how tools can read from session state.
def retrieve_userinfo(tool_context: ToolContext)-> Dict[str,Any]:
    """
    Tool to retrieve user name and country from session state.
    """

    #Read from session state
    user_name=tool_context.state.get("user:name","Username not found.")
    country=tool_context.state.get("user:country","Country not found")
    return {"status":"success","user_name":user_name,"country":country}

print("Tools created")

Tools created


#### Creating an Agent with Session State tools

In [76]:
#config
APP_NAME="default"
USER_ID="default"
MODEL_NAME="gemini-2.5-flash-lite"

root_agent=LlmAgent(
  model=Gemini(model=MODEL_NAME,retry_options=retry_config),
  name="text_chat_bot",
  description="""A text chatbot.
    Tools for managing user context:
    * To record username and country when provided use `save_userinfo` tool.
    * To fetch username and country when required use `retrieve_userinfo` tool.
    """,
    tools=[save_userinfo,retrieve_userinfo]
)

session_service = InMemorySessionService()
runner = Runner(agent=root_agent,session_service=session_service,app_name="default")

print("Agent with session state tools initialized!")

App name mismatch detected. The runner is configured with app name "default", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Agent with session state tools initialized!


In [77]:
await run_session(
  runner,
  [
    "Hi there, how are you doing today? What is my name?",# Agent shouldn't know the name yet
    "My name is Ilias, Im from Greece",# Provide name - agent should save it
    "What is my name? Where do i come from?"# Agent should recall from session state
  ],
  "state-demo"
)


 ### SESSION: state-demo

 User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing great. I'm not sure what your name is, though. Can you tell me? 


 User > My name is Ilias, Im from Greece


gemini-2.5-flash-lite >  Nice to meet you, Ilias! I've saved your information.

 User > What is my name? Where do i come from?


gemini-2.5-flash-lite >  Your name is Ilias and you come from Greece.


In [78]:
#inspecting session state

#retrieve the session and inspect its state

session = await session_service.get_session(
  app_name=APP_NAME,user_id=USER_ID, session_id="state-demo"
)

print("Session State Contents:")
print(session.state)
print("\n The 'user:name' and 'user:country' keys are storing the data.")

Session State Contents:
{'user:name': 'Ilias', 'user:country': 'Greece'}

 The 'user:name' and 'user:country' keys are storing the data.


In [122]:
session.events[1].content.parts[0].text

"Hello! I'm doing great. I'm not sure what your name is, though. Can you tell me? \n"

In [134]:
vars(session)

{'id': 'state-demo',
 'app_name': 'default',
 'user_id': 'default',
 'state': {'user:name': 'Ilias', 'user:country': 'Greece'},
 'events': [Event(model_version=None, content=Content(
    parts=[
      Part(
        text='Hi there, how are you doing today? What is my name?'
      ),
    ],
    role='user'
  ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-164cd67f-7dab-47cd-940a-866dabf3bc77', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=Non

In [135]:
# Start a completely new session - the agent won't know my name
await run_session(
    runner,
    ["Hi there, how are you doing today? What is my name?"],
    "new-isolated-session",
)

# Expected: The agent won't know the name because this is a different session


 ### SESSION: new-isolated-session

 User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing great, thank you for asking. I'd love to tell you your name, but I don't have access to that information right now. To do that, I'd need you to tell me your name and country first.



In [137]:
# Check the state of the new session
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="new-isolated-session"
)

print("New Session State:")
print(session.state)

# Note: Depending on implementation, you might see shared state here.
# This is where the distinction between session-specific and user-specific state becomes important.

#session_id => new session
#user_id => same person
#app_name => same App
# Its like you are the same user in the same app, you have this user state

New Session State:
{'user:name': 'Ilias', 'user:country': 'Greece'}
